# 1. Loader

In [1]:
from helper import *
from algorithms import *
import menpo3d
from importlib import reload
import datetime
import numpy as np
import importlib
from scipy import sparse
import time
import matplotlib.pyplot as plt
import warnings

# %matplotlib qt

First load the BFM mean model as the target mesh.

On shark, please comment on `%matplotlib qt` and all `mesh.view()` codes, as shark does not provides these viewings. If you are on your laptop, please un-comment these and help yourself viewing 3D faces.

In [2]:
#bfm_mean = loader.get_mean_model("../BFM.mat")
#bfm_mean.view()

You should see

<img src="./BFM_mean.png">

Then load the source mesh to do the transformation.

In [3]:
source = loader.get_mesh("../lsfm-ori/1/1_0001.obj")
#source.view()

You shall see
<img src="./source_mesh.png">

# 2. Test matrix norm solver

To be concise, suprress all warnings temporarily.

In [4]:
warnings.filterwarnings("ignore")

In [5]:
#importlib.reload(math_helper)

By default, we use `umfpack` solver.

In [6]:
A = np.random.random((4000,5000))
B = np.random.random((4000,300))
solver = math_helper.Solver(A, B)

start = time.time()
X = solver.solve()
error = np.sum((A @ X - B) ** 2)
print(error)

print("\n time spent: {:.1f} seconds".format(time.time() - start))

iter: 1, error: 400126.60
iter: 2, error: 0.00
1.0004106519311116e-17

 time spent: 30.2 seconds


On comparison, with naive solver.

In [71]:
A = np.random.random((4000,5000))
B = np.random.random((4000,300))
solver = math_helper.Solver(A, B, solver="naive")

start = time.time()
X = solver.solve()
error = np.sum((A @ X - B) ** 2)
print(error)

print("\n time spent: {:.1f} seconds".format(time.time() - start))

iter: 1, error: 400080.04
solving linear equation...
2018-11-12 14:31:20.359657
inversing matrix
2018-11-12 14:31:35.188524
iter: 2, error: 416857598.48
solving linear equation...
2018-11-12 14:31:37.813341
inversing matrix
2018-11-12 14:31:52.670348
iter: 3, error: 416857598.48
solving linear equation...
2018-11-12 14:31:55.261406
inversing matrix
2018-11-12 14:32:10.098824
iter: 4, error: 416857598.48
solving linear equation...
2018-11-12 14:32:12.706352
inversing matrix
2018-11-12 14:32:27.537560
iter: 5, error: 416857598.48
solving linear equation...
2018-11-12 14:32:30.128560
inversing matrix
2018-11-12 14:32:44.951544
iter: 6, error: 416857598.48
solving linear equation...
2018-11-12 14:32:47.561703
inversing matrix
2018-11-12 14:33:02.409826
iter: 7, error: 416857598.48
solving linear equation...
2018-11-12 14:33:05.005904
inversing matrix
2018-11-12 14:33:19.844723
iter: 8, error: 416857598.48
solving linear equation...
2018-11-12 14:33:22.436402
inversing matrix
2018-11-12 14:

| solver  | umfpack | naive |
|------|:------------:|----:|
| time | 30.2s | 174.6s |

# 3. Non-rigid icp

First, test non-rigid icp algorithm.

In [5]:
target = loader.get_mesh("../lsfm-ori/1/1_0002.obj")
#target.view()

In [7]:
fitter = no_landmark_nicp.NonRigidIcp(max_iter=5)
transformed_source, training_info = fitter.non_rigid_icp(source, bfm_mean)

Epoch 1 with stiffness 50
 - 1 regularized_error: 125643.643 	 0.000
 - 2 regularized_error: 125643.643 	 474.533
 - 3 regularized_error: 6.096 	 595.409
 - 4 regularized_error: 68.296 	 441.307
 - 5 regularized_error: 69.013 	 303.596
Epoch 2 with stiffness 20
 - 1 regularized_error: 18.755 	 575.332
 - 2 regularized_error: 39.527 	 681.874
 - 3 regularized_error: 34.161 	 471.699
 - 4 regularized_error: 13.974 	 339.109
 - 5 regularized_error: 17.149 	 284.134
Epoch 3 with stiffness 5
 - 1 regularized_error: 122.163 	 568.132
 - 2 regularized_error: 126.928 	 531.673
 - 3 regularized_error: 140.772 	 333.243
 - 4 regularized_error: 838.006 	 532.407
 - 5 regularized_error: 826.657 	 313.707
Epoch 4 with stiffness 2
 - 1 regularized_error: 112.611 	 488.828
 - 2 regularized_error: 136.500 	 416.373
 - 3 regularized_error: 2579.018 	 469.719
 - 4 regularized_error: 2584.025 	 613.301
 - 5 regularized_error: 966.479 	 777.046
Epoch 5 with stiffness 0.8
 - 1 regularized_error: 189.493 	 

After 2 epoches with each of 5 iterations, we could see
<img src="./face_after_5_iters.png">

The problem here is though, after **5 epoches**, with each of **5 iterations**, it's also this shape.

In [6]:
# fitter2 = no_landmark_nicp.NonRigidIcp(max_iter=5)
# transformed_source2, training_info2 = fitter.non_rigid_icp(source, target)

Now store the mesh.

In [10]:
import pickle as pkl

with open('tr.pkl', 'wb') as fw:
    pkl.dump({'points': transformed_source.points, 'trilist': transformed_source.trilist}, fw)

# Test Pipeline

First we test integration of `loading` and `aligning`.

In [9]:
from pipeline import Pipeline

In [14]:
p = Pipeline(base_model_path='../lsfm-ori/1/1_0001.obj', stiffness_weights=[2, 0.8], data_weights=[None, None], max_iter=2)


loading target mesh ../lsfm-ori/1/1_0001.obj



In [15]:
meshes, logs = p.align(input_path='../lsfm-ori/1')


loading mesh file ../lsfm-ori/1/1_0003.obj

Epoch 1 with stiffness 2
 - 1 loss: 0.001 regularized_loss: 0.000  
Epoch 2 with stiffness 0.8
 - 1 loss: 0.000 regularized_loss: 0.000  
{'regularized_loss': [2.924586148206691e-07, 8.403758925026211e-11], 'loss': [0.0008089405285939707, 2.3244797186622499e-07]}

loading mesh file ../lsfm-ori/1/1_0002.obj

Epoch 1 with stiffness 2
 - 1 loss: 0.000 regularized_loss: 0.000  
Epoch 2 with stiffness 0.8
 - 1 loss: 0.000 regularized_loss: 0.000  
{'regularized_loss': [9.35135004499074e-12, 1.1913059882980329e-09], 'loss': [2.5865834224444386e-08, 3.2951523636323587e-06]}

loading mesh file ../lsfm-ori/1/1_0001.obj

Epoch 1 with stiffness 2
 - 1 loss: 0.010 regularized_loss: 0.000  
Epoch 2 with stiffness 0.8
 - 1 loss: 0.000 regularized_loss: 0.000  
{'regularized_loss': [3.686248632189613e-06, 1.2021548179640807e-08], 'loss': [0.01019616371663647, 3.325160226488647e-05]}

loading mesh file ../lsfm-ori/1/1_0005.obj

Epoch 1 with stiffness 2
 - 1

Then we test on `PCA`s.

* First PCA on number of points.

In [20]:
pca_meshes = p.prune_on_num_points(meshes)

PCA error, max number of points is too large, use 7 points instead


Now for each mesh, the number of points should be at most the number of meshes.

In [35]:
pca_meshes[0].points.shape

(7, 3)

* Then PCA on number of meshes.

In [36]:
pca_model = p.pca_prune(pca_meshes)

Allocated data matrix of size 1.15 KB (7 samples)
Building data matrix: [==========] 100% (7/7) - done.                           
Retaining 99.70% of eigenvalues keeps 0.997 components


In [54]:
print("Final PCA Model:\n# of components: {}\n# of points for each mesh (3 dims total): {}\neigen value respective ratios: {}\
      \neigen value accumulative ratios: {}"
      .format(str(pca_model.components.shape[0]),
             str(pca_model.components.shape[1]),
             str(pca_model.eigenvalues_ratio()),
             str(pca_model.eigenvalues_cumulative_ratio())))

Final PCA Model:
# of components: 6
# of points for each mesh (3 dims total): 21
eigen value respective ratios: [0.65075597 0.18245646 0.08223279 0.03341399 0.02759639 0.02354441]      
eigen value accumulative ratios: [0.65075597 0.83321243 0.91544522 0.94885921 0.97645559 1.        ]
